In [1]:
import json
import os

manual_path = "../data/labels_manual.jsonl"
auto_path = "../data/labels_automatic/random_n3_scn1.jsonl"

golden = []
preds = []

with open(manual_path, "r") as f:
    for line in f:
        item = json.loads(line)
        if "template" in item["id"]: continue
        golden.append(item)
        
with open(auto_path, "r") as f:
    for line in f:
        preds.append(json.loads(line))

In [8]:
!pip install seqeval datasets

  Preparing metadata (setup.py) ... done
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.0 kB)
  Using cached propcache-0.2.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached yarl-1.18.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (69 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached huggingface_hub-0.28.1-py

In [2]:
import os
import json
import re
from transformers import AutoTokenizer
from datasets import Dataset

# Initialize a fast tokenizer (use any pretrained model you plan to use)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True)

In [3]:
name2id = {
    None: -100,
    "O": 0,    
}
for entity_type in golden[0]["entities"].keys():
    name2id[f"B-{entity_type}"] = len(name2id)-1
    name2id[f"I-{entity_type}"] = len(name2id)-1
id2name = {v: k for k, v in name2id.items()}

In [45]:
with open("../data/lyrics/{}.txt".format("American Kids_Kenny Chesney"), "r") as f:
    text = f.read()
    
encoding = tokenizer(
    [text*4, text*2], 
    max_length=512,
    truncation=True,
    stride=16,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding=True)

In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# 1. Setup: choose a tokenizer and model for token classification.
model_name = "distilbert-base-uncased"  # Demo model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.eval()  # We'll do a simple inference (no gradient needed)

# 2. Example texts (make them long to illustrate overflow).
texts = [
    "Hugging Face Inc. is a company based in New York City. " * 5 + 
    "They are the creators of the Transformers library for natural language processing.",
    "OpenAI is an AI research lab consisting of the for-profit OpenAI LP " * 4 +
    "and its parent company, the non-profit OpenAI Inc."
]

# 3. Tokenize with overflow
# - max_length: the chunk size
# - stride: overlap between consecutive chunks (in tokens)
# - return_overflowing_tokens: produce multiple chunks instead of truncating
# - return_offsets_mapping: map each token to (start_char, end_char) in the original text
encoding = tokenizer(
    texts,
    max_length=32,
    truncation=True,
    stride=8,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding=True
)

# encoding is a BatchEncoding. The important fields:
#   - input_ids: list of list (each sublist is one chunk)
#   - attention_mask
#   - offset_mapping: list of list, specifying char-level offsets for each token
#   - overflow_to_sample_mapping: which original sample each chunk corresponds to

input_ids = encoding["input_ids"]
attention_masks = encoding["attention_mask"]
offset_mappings = encoding["offset_mapping"]
sample_map = encoding["overflow_to_sample_mapping"]

# 4. Run model inference chunk by chunk.
# Convert lists to tensors
input_ids_t = torch.tensor(input_ids)
attention_masks_t = torch.tensor(attention_masks)

with torch.no_grad():
    outputs = model(input_ids_t, attention_mask=attention_masks_t)
    # For token classification, model outputs logits of shape [batch_size, seq_len, num_labels]
    logits = outputs.logits  # (num_chunks, seq_len, num_labels)

# 5. Combine chunk-level predictions back into full-sequence predictions
# Prepare a structure to collect predictions for each original sample.
num_original_texts = len(texts)
all_predictions = [[] for _ in range(num_original_texts)]

# We’ll loop over each chunk, find which original text it belongs to,
# and map the predictions back via offsets.
for chunk_idx, chunk_logits in enumerate(logits):
    sample_idx = sample_map[chunk_idx]  # original sample this chunk belongs to
    offsets = offset_mappings[chunk_idx]

    # chunk_logits shape: (seq_len, num_labels)
    for token_pos, token_logits in enumerate(chunk_logits):
        start_char, end_char = offsets[token_pos]
        
        # Skip special tokens ([CLS], [SEP], etc.) which often have offset (0,0)
        if start_char == 0 and end_char == 0:
            continue
        
        # Get predicted label ID (argmax over the label dimension)
        predicted_label_id = torch.argmax(token_logits).item()

        # Store the prediction in a list, along with the character span
        # You might store the actual label string if you have a label map.
        all_predictions[sample_idx].append({
            "char_span": (start_char, end_char),
            "label_id": predicted_label_id
            # "label_name": label_list[predicted_label_id] if you have label_list
        })

# 6. Dealing with overlaps
# Because we used a stride, the same text span may appear in multiple chunks.
# You can decide how to reconcile conflicting predictions in overlaps:
#   - Use the prediction from the chunk where it's centered
#   - Use the first or last chunk's prediction
#   - Or do a vote/average among overlapping predictions
# Below is a trivial example that doesn't do deduplication, so each token
# might appear multiple times if it overlapped. In a real project, you'd
# implement your overlap strategy carefully.

# Let's print out combined predictions (naively) for each original text
for idx, preds in enumerate(all_predictions):
    print(f"\n=== Predictions for text {idx} ===")
    for p in preds[:15]:  # just show the first 15
        char_span = p["char_span"]
        label_id = p["label_id"]
        snippet = texts[idx][char_span[0]:char_span[1]]
        print(f"Text span: '{snippet}', Label ID: {label_id}")

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: list indices must be integers or slices, not tuple

In [ ]:
def create_bio_tags(text, labels_dict):
    """
    Given a text string and a labels dictionary of the form:
       {"class1": [entity1, entity2, ...], "class2": [entity3, ...], ...}
    this function returns the tokenized text and a list of BIO tags.
    """
    # Collect all entity spans with their class label
    # Each span is represented as a dictionary with start/end offsets.
    entity_spans = []
    for label, entities in labels_dict.items():
        for entity in entities:
            # Use re.finditer to locate all occurrences of the entity in the text.
            # re.escape is used to avoid issues with special characters.
            for match in re.finditer(re.escape(entity), text):
                span = {"label": label, "start": match.start(), "end": match.end()}
                entity_spans.append(span)
    # Sort spans by start position (this helps if there are multiple entities)
    entity_spans = sorted(entity_spans, key=lambda x: x["start"])

    # Tokenize the text using the fast tokenizer to get offset mappings
    encoding = tokenizer(text, return_offsets_mapping=True, truncation=False, padding=True)
    tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
    offset_mapping = encoding["offset_mapping"]
    
    # Initialize all tags as "O" (outside)
    bio_tags = ["O" if token not in tokenizer.all_special_tokens else None for token in tokens] * len(tokens)

    # For each entity, label the tokens whose offsets overlap with the entity span.
    for entity in entity_spans:
        ent_start = entity["start"]
        ent_end = entity["end"]
        entity_label = entity["label"]
        token_indices = []

        # Loop over the tokens’ offset mappings.
        for i, (tok_start, tok_end) in enumerate(offset_mapping):
            # Some fast tokenizers return (0, 0) for special tokens (e.g., [CLS], [SEP])
            if tok_start == tok_end == 0:
                continue
            # If the token lies completely after the entity span, we can break early.
            if tok_start >= ent_end:
                break
            # If the token overlaps with the entity span, record its index.
            if tok_end > ent_start and tok_start < ent_end:
                token_indices.append(i)
        
        # Assign BIO tags: first token gets "B-<label>", subsequent tokens get "I-<label>".
        if token_indices:
            bio_tags[token_indices[0]] = f"B-{entity_label}"
            for idx in token_indices[1:]:
                bio_tags[idx] = f"I-{entity_label}"
                
    ner_tags = [name2id[tag] for tag in bio_tags]
    encoding["labels"] = ner_tags
    del encoding["offset_mapping"]
    
    return tokens, bio_tags, encoding

In [19]:
golden_labels = {g['id']: g['entities'] for g in golden}

# List to hold all examples.
data = []
encodings = []

# Loop over files in the folder.
# Here we assume text files have extension ".txt" and labels are in a JSON file with the same basename.
for filename in os.listdir("../data/tmp/labeled"):
    if filename.endswith(".txt"):
        txt_path = os.path.join("../data/tmp/labeled", filename)
        
        # Read the text.
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        labels_dict = golden_labels[filename.split('.')[0]]
        
        # Create tokens and BIO tags.
        encoding = create_bio_tags(text, labels_dict)
        encodings.append(encoding)

Token indices sequence length is longer than the specified maximum sequence length for this model (1260 > 512). Running this sequence through the model will result in indexing errors


In [6]:
from datasets import DatasetDict

# Split the dataset into train, validation, and test sets
dataset = Dataset.from_list(encodings)
train_test_split = dataset.train_test_split(test_size=0.2)
test_valid_split = train_test_split['test'].train_test_split(test_size=0.5)

# Combine the splits into a DatasetDict
tokenized_datasets = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test']
})

# Verify the splits
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 128
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 16
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 16
    })
})


In [7]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [146]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)


In [8]:
import evaluate

metric = evaluate.load("seqeval")

In [9]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2name[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2name[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [10]:
labels = tokenized_datasets["train"][0]["labels"]
# labels = tokenized_datasets["train"][8]["labels"]
labels = [id2name[i] for i in labels if i != -100]

predictions = labels.copy()
predictions[10] = "B-human-power"

metric.compute(predictions=[labels], references=[predictions])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/artempris/miniconda3/envs/mobility/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'human-power': {'precision': np.float64(0.0),
  'recall': np.float64(0.0),
  'f1': np.float64(0.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(0.0),
 'overall_recall': np.float64(0.0),
 'overall_f1': np.float64(0.0),
 'overall_accuracy': 0.9999961703431373}

In [11]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2name[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2name[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [12]:
id2label = {i: name for i, name in id2name.items() if i and name}
label2id = {name: i for name, i in name2id.items() if i and name}

In [44]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model.config.num_labels

12

In [15]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
)

/Users/artempris/miniconda3/envs/mobility/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/var/folders/db/z1j0x3f51c56c00nhhmhl7qw0000gn/T/ipykernel_96747/3203677919.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: expected sequence of length 262144 at dim 1 (got 126025)

In [34]:
with open("../data/lyrics/{}.txt".format("American Kids_Kenny Chesney"), "r") as f:
    text = f.read()

In [35]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification", model="bert-finetuned-ner", aggregation_strategy="simple"
)
res = token_classifier(text)
res

OSError: bert-finetuned-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`